In [105]:
# Импортируем библиотеки и модули
import pandas as pd  
from sklearn.model_selection import train_test_split # Загружаем специальный инструмент для разбивки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели
from sklearn.feature_selection import chi2 # хи-квадрат
from sklearn.feature_selection import f_classif # anova
from sklearn.feature_selection import SelectKBest
from collections import Counter

# <center>1. Знакомство с данными<center>

In [120]:
# Загружаем данные
hotels = pd.read_csv('C:\IDE\data\Project-3\hotels.csv')
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


In [107]:
# Выводим количество пропущенных значений в столбцах
colls_null_percent = hotels.isnull().mean()*100
colls_with_null = colls_null_percent[colls_null_percent>0].sort_values(ascending=False)
display(colls_with_null)

lat    0.63288
lng    0.63288
dtype: float64

In [108]:
# Выводим информацию о таблице
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

# <center>2. Подготовка данных<center>

In [109]:
# Выводим информацию о числе уникальных названий отелей(количество самих отелей в данных)
display(len(hotels['hotel_name'].unique()))

1492

In [110]:
# Преобразуем дату в формат datetime
hotels['review_date'] = pd.to_datetime(hotels['review_date'], yearfirst=True)

# Выводим дату самого раннего и самого свежего отзыва
print(hotels['review_date'].min())
print(hotels['review_date'].max())

2015-08-04 00:00:00
2017-08-03 00:00:00


In [111]:
# Заменяем элементы в строке признака
replace_1 = lambda x: x.replace('[', '')
hotels['tags']= hotels['tags'].apply(replace_1)

replace_2 = lambda x: x.replace(']', '')
hotels['tags']= hotels['tags'].apply(replace_2)

# делим строку признака на список отдельных тегов
separate_func = lambda x: x.split(', ')
hotels['tags']= hotels['tags'].apply(separate_func)

In [112]:
# Создаем функцию для подсчета уникальных значений тегов
def unique_tags(column):
    """Функция для создания списка уникальных тегов

    Args:
        column (object): На вход подаеться признак с тегами, оставленными постояльцами отеля

    Returns:
        list: Список уникальных значений тегов
    """
    tags_list = list()
    
    for elem in column:
        for i in elem:
            if i not in tags_list:
                tags_list.append(i)
                
    return tags_list
print(len(unique_tags(hotels['tags'])))

2368


In [113]:
# Находим самый популярный тэг
def all_tags(column):
    """Функция для создания списка всех тегов

    Args:
        column (object): На вход подается признак с тегами, оставленными постояльцами отеля

    Returns:
        list: Список всех тегов
    """
    tags_list = list()
    
    for elem in column:
        for i in elem:
            tags_list.append(i)
            
    return tags_list
# Используем обьект Counter для подсчета количества каждого тега
c = Counter(all_tags(hotels['tags']))  
print(c)      

Counter({"' Leisure trip '": 313593, "' Submitted from a mobile device '": 230778, "' Couple '": 189212, "' Stayed 1 night '": 145373, "' Stayed 2 nights '": 100263, "' Solo traveler '": 81235, "' Stayed 3 nights '": 72000, "' Business trip '": 61989, "' Group '": 49088, "' Family with young children '": 45836, "' Stayed 4 nights '": 35748, "' Double Room '": 26386, "' Standard Double Room '": 24151, "' Superior Double Room '": 23550, "' Family with older children '": 19802, "' Deluxe Double Room '": 18623, "' Double or Twin Room '": 16824, "' Stayed 5 nights '": 15611, "' Standard Double or Twin Room '": 13062, "' Classic Double Room '": 12716, "' Superior Double or Twin Room '": 10238, "' 2 rooms '": 9287, "' Stayed 6 nights '": 7399, "' Standard Twin Room '": 7325, "' Single Room '": 7227, "' Twin Room '": 6279, "' Stayed 7 nights '": 5549, "' Executive Double Room '": 4763, "' Classic Double or Twin Room '": 4576, "' Superior Twin Room '": 4540, "' Club Double Room '": 4485, "' Del

In [114]:
# Находим на сколько всего ночей, чаще всего, останавливаются потушественники в отелях
def night_stay(column):
    """Функция для создания списка тегов с количеством ночей, на которые останавливались постояльцы

    Args:
        column (object): На вход подаеться признак с тегами, оставленными постояльцами отеля

    Returns:
        list: Список с тегами о количестве ночей. проведенных в отеле постояльцами
    """
    night_stay_list = list()
    
    for elem in column:
        for i in elem:
            if 'Stayed' in i:
                night_stay_list.append(i)
                
    return night_stay_list
# Используем обьект Counter для подсчета каждого тега в списке
c = Counter(night_stay(hotels['tags']))
print(c)             

Counter({"' Stayed 1 night '": 145373, "' Stayed 2 nights '": 100263, "' Stayed 3 nights '": 72000, "' Stayed 4 nights '": 35748, "' Stayed 5 nights '": 15611, "' Stayed 6 nights '": 7399, "' Stayed 7 nights '": 5549, "' Stayed 8 nights '": 1910, "' Stayed 9 nights '": 966, "' Stayed 10 nights '": 663, "' Stayed 11 nights '": 306, "' Stayed 12 nights '": 217, "' Stayed 14 nights '": 184, "' Stayed 13 nights '": 174, "' Stayed 15 nights '": 87, "' Stayed 16 nights '": 38, "' Stayed 17 nights '": 27, "' Stayed 18 nights '": 24, "' Stayed 19 nights '": 23, "' Stayed 21 nights '": 19, "' Stayed 20 nights '": 17, "' Stayed 27 nights '": 10, "' Stayed 30 nights '": 10, "' Stayed 22 nights '": 8, "' Stayed 28 nights '": 7, "' Stayed 26 nights '": 6, "' Stayed 23 nights '": 6, "' Stayed 24 nights '": 5, "' Stayed 25 nights '": 4, "' Stayed 29 nights '": 3})


# <center>3. Преобразование признаков<center>

### *3.1 Работа с признаком hotel_address. Создадим новый признак - страна отеля*

In [122]:
# Для начала найдем в каких странах находятся отели и заодно посмотрим количество отзывов для каждой страны
def address_hotel(column):
    """Функция для создания списка стран в которых распологаются отели

    Args:
        column (object): На вход подается признак с адресом отеля

    Returns:
        list: Список всех стран, в которых находятся отели
    """
    address_list = list()
    for elem in column:
        elem = elem.split(' ')
        elem = elem[-2:]
        
        if 'United' in elem:
           elem = elem[0] + ' ' + elem[1]
           address_list.append(elem)
        else:
            address_list.append(elem[-1])
               
    return address_list
# С помощью обьекта Counter смотрим на уникальные значения стран и количество отзывов об отелях 
c = Counter(address_hotel(hotels['hotel_address']))
print(c)  

# Создадим функцию для признака страны
def hotel_country(column):
    """Функция для создания признака страны

    Args:
        column (object): На вход подается признак с адресом отеля

    Returns:
        str: Строка с названием страны, где расположен отель
    """
    column_splited = column.split(' ')[-2:]
    if 'United' in column_splited:
       return column_splited[0] + ' ' + column_splited[1]
    else:
       return column_splited[-1]  
             
# Создаем новый признак, применяя функцию к признаку адреса отеля   
hotels['hotel_country'] = hotels['hotel_address'].apply(hotel_country)
# Удаляем признак с адрессом отеля, так он содержит слишком много уникальных значений 
hotels = hotels.drop(['hotel_address'], axis=1)

Counter({'United Kingdom': 196774, 'Spain': 45132, 'France': 44830, 'Netherlands': 43006, 'Austria': 29178, 'Italy': 27883})


### *3.2 Работа с признаком review_date. Создадим два целочисленных признака: Год и Месяц.*

In [125]:
display(hotels)

,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng,hotel_country
0,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671,United Kingdom
1,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097,United Kingdom
2,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643,France
3,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540,France
4,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060,Netherlands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386798,107,4/19/2017,9.0,Hotel Moonlight,France,No Negative,0,617,Tr s proche du metro Earl s court,10,10,8.8,"[' Leisure trip ', ' Group ', ' Club Double or...",106 day,51.494028,-0.191050,United Kingdom
386799,272,2/13/2017,8.4,BEST WESTERN PLUS Amedia Wien,Turkey,No Negative,0,3224,The bed was so comfy I stayed with my boyfrie...,93,1,9.2,"[' Leisure trip ', ' Couple ', ' Standard Doub...",171 day,48.192379,16.399451,Austria
386800,457,2/7/2016,6.8,Bloomsbury Palace Hotel,Netherlands,room is really small but guess is normal in L...,12,2751,great location simple check in out nice shower,9,21,8.3,"[' Business trip ', ' Solo traveler ', ' Singl...",543 day,51.520795,-0.131084,United Kingdom
386801,365,5/21/2017,8.1,The Marble Arch London,United Arab Emirates,No Negative,0,1567,Location and very comfy bed,6,28,9.2,"[' Leisure trip ', ' Solo traveler ', ' Deluxe...",74 days,51.515125,-0.160066,United Kingdom


In [ ]:
# Непрерывные признаки
num_cols = ['total_number_of_reviews', 'review_total_negative_word_counts', 'additional_number_of_scoring', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given']

# Категориальные признаки
cat_cols = ['average_score', 'lat', 'lng', 'days_since_review', 'tags', 'positive_review', 'negative_review', 'reviewer_nationality', 'hotel_name', 'review_date', 'hotel_address']

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score']

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))